In [1]:
#modified from https://github.com/floydhub/quick-start-pytorch/blob/master/mnist.ipynb

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim
import torchvision

from torch.autograd import Variable
cuda = torch.cuda.is_available()

torch.manual_seed(0)
if cuda:
    torch.cuda.manual_seed(0)

In [2]:
train = torchvision.datasets.MNIST('./data', train=True, download=True, transform=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

test = torchvision.datasets.MNIST('./data', train=False, download=True, transform=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

# Create DataLoader
dataloader_args = dict(shuffle=True, batch_size=10000,num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
train_loader = dataloader.DataLoader(train, **dataloader_args)
test_loader = dataloader.DataLoader(test, **dataloader_args)

train_data = train.train_data
train_data = train.transform(train_data.numpy())

In [3]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc = nn.Linear(784, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        x = x.view((-1, 784))
        h = F.relu(self.fc(x))
        h = self.fc2(h)
        return F.log_softmax(h, dim=1)    
    
model = Model()
if cuda:
    model.cuda()
    
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [4]:
EPOCHS = 20
losses = []

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        
        if cuda:
            data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        y_pred = model(data) 

        loss = F.cross_entropy(y_pred, target)
        losses.append(loss.cpu().data.item())
        loss.backward()
        optimizer.step()

        #if batch_idx % 100 == 1:
        print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1,
            EPOCHS,
            batch_idx * len(data), 
            len(train_loader.dataset),
            100. * batch_idx / len(train_loader), 
            loss.cpu().data.item()))

    evaluate_x = Variable(test_loader.dataset.test_data.type_as(torch.FloatTensor()))
    evaluate_y = Variable(test_loader.dataset.test_labels)
    if cuda:
        evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()

    model.eval()
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    d = pred.eq(evaluate_y.data).cpu()
    accuracy = d.sum().item()*1./d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Test Accuracy: {:.2f}%'.format(
        epoch+1,
        EPOCHS,
        len(train_loader.dataset), 
        len(train_loader.dataset),
        100. * batch_idx / len(train_loader), 
        loss.cpu().data.item(),
        accuracy*100.0))

 Train Epoch: 1/20 [0/60000 (0%)]	Loss: 2.301791
 Train Epoch: 1/20 [10000/60000 (17%)]	Loss: 2.281543
 Train Epoch: 1/20 [20000/60000 (33%)]	Loss: 2.262036
 Train Epoch: 1/20 [30000/60000 (50%)]	Loss: 2.243205
 Train Epoch: 1/20 [40000/60000 (67%)]	Loss: 2.222202
 Train Epoch: 1/20 [50000/60000 (83%)]	Loss: 2.202651
 Train Epoch: 1/20 [60000/60000 (83%)]	Loss: 2.202651	 Test Accuracy: 51.54%
 Train Epoch: 2/20 [0/60000 (0%)]	Loss: 2.180925
 Train Epoch: 2/20 [10000/60000 (17%)]	Loss: 2.163620
 Train Epoch: 2/20 [20000/60000 (33%)]	Loss: 2.141855
 Train Epoch: 2/20 [30000/60000 (50%)]	Loss: 2.122058
 Train Epoch: 2/20 [40000/60000 (67%)]	Loss: 2.102757
 Train Epoch: 2/20 [50000/60000 (83%)]	Loss: 2.082337
 Train Epoch: 2/20 [60000/60000 (83%)]	Loss: 2.082337	 Test Accuracy: 68.20%
 Train Epoch: 3/20 [0/60000 (0%)]	Loss: 2.060444
 Train Epoch: 3/20 [10000/60000 (17%)]	Loss: 2.045146
 Train Epoch: 3/20 [20000/60000 (33%)]	Loss: 2.022069
 Train Epoch: 3/20 [30000/60000 (50%)]	Loss: 2.0005